In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk.corpus
import re, string
import progressbar
import numpy as np
import matplotlib.gridspec as gridspec 

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, recall_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ML Project - Quora/Data/train.csv')
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
train = train.drop('qid', axis=1)
train.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [ ]:
train.isnull().sum()

question_text    0
target           0
dtype: int64

In [ ]:
# cleaning the data
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('omw-1.4')
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    text = nopunct.split(' ')
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", str(text)) # remove unicode characters
    text = re.sub(r'[^a-zA-Z\s]+', '', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop]) # remove stop words
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')]) # lemmatization
    text = re.sub(' +', ' ', text) # remove extra spaces
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
tqdm.pandas()
train_clean = train
train_clean['clean_text'] = train['question_text'].progress_apply(clean_text)

100%|██████████| 1306122/1306122 [02:46<00:00, 7856.79it/s]


In [ ]:
train_clean.head()


,question_text,target,clean_text
0,How did Quebec nationalists see their province...,0,quebec nationalist see province nation
1,"Do you have an adopted dog, how would you enco...",0,adopted dog would encourage people adopt shop
2,Why does velocity affect time? Does velocity a...,0,velocity affect time velocity affect space geo...
3,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke used magdeburg hemisphere
4,Can I convert montra helicon D to a mountain b...,0,convert montra helicon mountain bike changing ...


In [ ]:
train['question_text'].iloc[0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [ ]:
train_clean['clean_text'].iloc[0]

'quebec nationalist see province nation '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/ML Project - Quora/Data/train_clean.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  train_clean.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
